# RK4 Advection Solver

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
import fd_tools as fd

## Introduction

In this notebook, we'll experiment with an RK4-based numerical solver for the 1D advection problem.  Here's the derivative matrix function from last notebook.

In [ ]:
def first_deriv_matrix(x, w):
    
    I, J, data = [], [], []
    n = len(x)
    
    # bias to the right until we can center
    p = (2*w+1)
    for i in range(w):
        I.extend(p*[i])
        J.extend(range(p))
        data.extend(fd.fd_coeff(1, x[i], x[:p]))
        
    # now fill in centered differences
    for i in range(w, n-w-1):
        I.extend(p*[i])
        J.extend(range(i-w, i+w+1))
        data.extend(fd.fd_coeff(1, x[i], x[i-w:i+w+1]))

    # bias to the left when we can't center anymore
    for i in range(n-w-1, n):
        I.extend(p*[i])
        J.extend(range(n-p, n))
        data.extend(fd.fd_coeff(1, x[i], x[n-p:]))
        
    return sp.csr_matrix((data, (I, J)))

## RK4

The RK4 method for the ODE $u'=f(u,t)$ takes the form

\begin{align*}
k_1 &= \Delta t f(U^j,t_j) & V_1 &= U^j + \frac{1}{2}k_1\\
k_2 &= \Delta t f\left(V^1, t_j+\frac{\Delta t}{2}\right) & V_2 &= U^j + \frac{1}{2}k_2\\
k_3 &= \Delta t f\left(V^2, t_j+\frac{\Delta t}{2}\right) & V_3 &= U^j + k_3\\
k_4 &= \Delta t f\left(V^3, t_j+\Delta t\right)\\
U^{j+1} &= U^j + \frac{1}{6}\left(k_1 + 2k_2 + 2k_3 + k_4\right)
\end{align*}

**NOTE:** I don't always compute the terms $V_1$, $V_2$ and $V_3$ explicitly, but they'll come in handy later in this notebook.

In [ ]:
def solve_advection_RK4(a, D, nx, dx, g, tf, cfl, n_ret):
    
    t_ret = []
    u_ret = []
    
    # target step size
    dt = cfl*(dx/a)

    # number of time steps
    nt = int(np.ceil(tf/dt))
    
    # fix time step
    dt = tf/nt
    
    # initial data
    u = np.zeros(nx)

    t_ret.append(0)
    u_ret.append(u.copy())
    
    ret_every = int(nt/n_ret)
    
    # iterate
    for it in range(1,nt+1):
        
        k1 = -dt*a*D*u
        V1 = u + k1/2
        k2 = -dt*a*D*(V1)
        V2 = u + k2/2
        k3 = -dt*a*D*(V2)
        V3 = u + k3
        k4 = -dt*a*D*(V3)
        u += (k1 + 2*k2 + 2*k3 + k4)/6
        
        t = it*dt
        u[0] = g(t)

        if it%ret_every == 0:
            t_ret.append(t)
            u_ret.append(u.copy())
        
    return t_ret, u_ret

### Boundary conditions

We'll start with zero initial data and have a wiggly pulse come in through the left boundary.

In [ ]:
def boundary(t):
    """
    """
    t0 = 2
    a = 10
    p = 6
    b = 6*np.pi
    return np.exp(-a*(t-t0)**p)*np.cos(b*t)

def boundary_demo():
    t = np.linspace(0, 4, 1000)
    plt.plot(t, boundary(t))
boundary_demo()


### Demo solver

Here's an example solver.  You can play around with this and try to see if the CFL condition suggested by the absolute stability analysis seems to work.  (Hint: It doesn't.)

In [ ]:
def solver_demo():
    a = 1.8
    xl, xu = 0, 18
    nx = 1000
    dx = (xu-xl)/(nx-1)
    x = np.linspace(xl, xu, nx)
    tf = 10

    D = first_deriv_matrix(x, 1)

    T, U = solve_advection_RK4(a, D, nx, dx, boundary, tf, 2.4, 5)

    for t,u in zip(T, U):
        plt.figure()
        plt.plot(x, boundary(t - x/a), color='#aaaaaa')
        plt.plot(x, u)
        plt.title('t = {:.3f}'.format(t))
        plt.ylim([-1,1])
        
solver_demo()

### Fixing the boundary conditions

The problem with stability is that we're not enforcing the correct intermediate boundary conditions.  Several ways to correct this can be found in [this paper](https://epubs.siam.org/doi/10.1137/0916072).  Below is one implementation.  It requires the exact evaluation of several time derivatives of the boundary value function.

In [ ]:
def boundary_derivs(t):
    """
    Found using Mathematica
    """
    t0 = 2
    a = 10
    p = 6
    b = 6*np.pi
    gp = (-(a*p*(t - t0)**p*np.cos(b*t)) + b*(-t + t0)*np.sin(b*t))*(np.exp(-a*(t - t0)**p)/(t - t0))
    gpp =  ((-(b**2*(t - t0)**2) + a*p*(1 - p + a*p*(t - t0)**p)*(t - t0)**p)*np.cos(b*t) + \
             2*a*b*p*(t - t0)**(1 + p)*np.sin(b*t))*(np.exp(-a*(t - t0)**p)/(t - t0)**2)
    gppp =  (-(a*p*(2 + 3*p*(-1 + a*(t - t0)**p) + \
               p**2*(1 - 3*a*(t - t0)**p + a**2*(t - t0)**(2*p)) - 3*b**2*(t - t0)**2)* \
             (t - t0)**p*np.cos(b*t)) + \
          b*(b**2*(t - t0)**2 - 3*a*p*(1 + p*(-1 + a*(t - t0)**p))*(t - t0)**p)*(t - t0)* \
           np.sin(b*t))*(np.exp(-a*(t - t0)**p)/(t - t0)**3)
    
    return gp, gpp, gppp
                                                                     


def solve_advection_RK4_fixed(a, D, nx, dx, g, g_derivs, tf, cfl, n_ret):
    
    t_ret = []
    u_ret = []
    
    # target step size
    dt = cfl*(dx/a)

    # number of time steps
    nt = int(np.ceil(tf/dt))
    
    # fix time step
    dt = tf/nt
    
    # initial data
    u = np.zeros(nx)

    t_ret.append(0)
    u_ret.append(u.copy())
    
    ret_every = int(nt/n_ret)
    
    # iterate
    for it in range(1,nt+1):
        
        # get boundary conditions and derivatives
        t = (it-1)*dt
        gp, gpp, gppp = g_derivs(t)
        
        k1 = -dt*a*D*u
        V1 = u + k1/2
        V1[0] = u[0] + (dt/2)*gp
        
        k2 = -dt*a*D*(V1)
        V2 = u + k2/2
        V2[0] = V1[0] + (dt**2)/4*gpp
        
        k3 = -dt*a*D*(V2)
        V3 = u + k3
        V3[0] = V2[0] + (dt**3)/4*gppp
        
        k4 = -dt*a*D*(V3)
        u += (k1 + 2*k2 + 2*k3 + k4)/6
        u[0] = g(t+dt)

        if it%ret_every == 0:
            t_ret.append(t)
            u_ret.append(u.copy())
        
    return t_ret, u_ret

In [ ]:
def fixed_solver_demo():
    a = 1.8
    xl, xu = 0, 18
    nx = 1000
    dx = (xu-xl)/(nx-1)
    x = np.linspace(xl, xu, nx)
    tf = 10

    D = first_deriv_matrix(x, 1)

    T, U = solve_advection_RK4_fixed(a, D, nx, dx, boundary, boundary_derivs, tf, 2.4, 5)

    for t,u in zip(T, U):
        plt.figure()
        plt.plot(x, boundary(t - x/a), color='#aaaaaa')
        plt.plot(x, u)
        plt.title('t = {:.3f}'.format(t))
        plt.ylim([-1,1])
        
fixed_solver_demo()